In [1]:
import pyspark as ps    # for the pyspark suite
import pandas as pd

In [2]:
spark = (ps.sql.SparkSession
         .builder
         .master('local[*]')
         .appName('lecture')
         .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
spark


In [4]:
df = spark.read.option("inferschema", "true").csv('data/patent.tsv',
                    header=True,       # use headers or not
                    quote='"',         # char for quotes
                    sep="\t")          # char for separation  

# prints the schema
df.printSchema()

# some functions are still valid
print("line count: {}\n".format(df.count()))

# show the table in a oh-so-nice format
df.show()

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- number: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- abstract: string (nullable = true)
 |-- title: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- num_claims: integer (nullable = true)
 |-- filename: string (nullable = true)
 |-- withdrawn: string (nullable = true)

line count: 7144425

+--------+-------+--------+-------+-------------------+--------------------+--------------------+----+----------+-------------+---------+
|      id|   type|  number|country|               date|            abstract|               title|kind|num_claims|     filename|withdrawn|
+--------+-------+--------+-------+-------------------+--------------------+--------------------+----+----------+-------------+---------+
|10000000|utility|10000000|     US|2018-06-19 00:00:00|A frequency modul...|Coherent LADAR us...|  B2|        20|ipg180619.xml|     NULL

In [20]:
df.select("type").distinct().show(20,False)

+--------------------------------+
|type                            |
+--------------------------------+
|defensive publication           |
|utility                         |
|plant                           |
|reissue                         |
|statutory invention registration|
|TVPP                            |
|design                          |
+--------------------------------+



In [16]:
df.select("id").distinct().show()

7144425

In [26]:
df.groupby(df.withdrawn).count().show()

+---------+-------+
|withdrawn|  count|
+---------+-------+
|        0|6963631|
|        1|   9668|
|     NULL| 171126|
+---------+-------+



In [27]:
df.groupby(df.country).count().show()

+-------+-------+
|country|  count|
+-------+-------+
|     US|7144425|
+-------+-------+



In [28]:
df.groupby(df.kind).count().show()

+----+-------+
|kind|  count|
+----+-------+
|  I5|      3|
|  H1|    273|
|  P2|  12558|
|  B2|3464834|
|   E|   8692|
|  I4|    509|
|  P3|   6052|
|  B1| 620301|
|   A|2391379|
|   S| 209582|
|  E1|  10107|
|   P|   8282|
|  S1| 409868|
|   H|   1982|
|  H2|      3|
+----+-------+



In [33]:
df.groupby(df.type).count().show(99,False)

+--------------------------------+-------+
|type                            |count  |
+--------------------------------+-------+
|defensive publication           |509    |
|utility                         |6476514|
|plant                           |26892  |
|reissue                         |18799  |
|statutory invention registration|2258   |
|TVPP                            |3      |
|design                          |619450 |
+--------------------------------+-------+



In [5]:
#df_panda = pd.DataFrame(df.take(1000), columns=df.columns)

In [9]:
from pyspark.sql.functions import min, max
min_date, max_date = df.select(min("date"), max("date")).first()
min_date, max_date


(datetime.datetime(1976, 1, 6, 0, 0), datetime.datetime(2019, 10, 8, 0, 0))

In [11]:
df_cpc = spark.read.option("inferschema", "true").csv('data/cpc_current.tsv',
                    header=True,       # use headers or not
                    quote='"',         # char for quotes
                    sep="\t")          # char for separation  

# prints the schema
df_cpc.printSchema()

# some functions are still valid
print("line count: {}\n".format(df_cpc.count()))

# show the table in a oh-so-nice format
df_cpc.show()

root
 |-- uuid: string (nullable = true)
 |-- patent_id: integer (nullable = true)
 |-- section_id: string (nullable = true)
 |-- subsection_id: string (nullable = true)
 |-- group_id: string (nullable = true)
 |-- subgroup_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sequence: integer (nullable = true)

line count: 39915464

+--------------------+---------+----------+-------------+--------+--------------+-----------+--------+
|                uuid|patent_id|section_id|subsection_id|group_id|   subgroup_id|   category|sequence|
+--------------------+---------+----------+-------------+--------+--------------+-----------+--------+
|000016xombd5lbk9l...|  7070831|         H|          H01|    H01L|H01L2924/01013| additional|      22|
|000070runw99gxjki...|  7618693|         C|          C09|    C09D|     C09D11/30|inventional|       1|
|00008erwm5297s6wv...|  8488869|         G|          G06|    G06T|G06T2207/10016| additional|      20|
|00008q01v2ziacpr0...|  9

In [43]:
ddf = df.join(df_cpc, df.id == df_cpc.patent_id).drop('id')

In [16]:
ddf.count()

39915464

In [17]:
df_cpc.count()

39915464

In [20]:
df_cpc.select("patent_id").distinct().count()

6452079

In [25]:
df_cpc.groupby(df_cpc.patent_id).count().show()

+---------+-----+
|patent_id|count|
+---------+-----+
|  6973866|   12|
|  9122810|   16|
|  7510754|   20|
|  9390338|   14|
|  9198479|   27|
|  9791123|    5|
|  5585105|   14|
|  6121532|   22|
|  5122749|    2|
|  4359781|    4|
|  9314865|    6|
|  7817829|    2|
| 10065879|   15|
|  6096781|    7|
|  6262027|   10|
|  7394163|   67|
|  5340740|    5|
| 10275120|    3|
|  7265388|   13|
|  8859658|   18|
+---------+-----+
only showing top 20 rows



In [28]:
df_cpc.filter(df_cpc.patent_id==6121532).sort(df_cpc.sequence.asc()).show()

+--------------------+---------+----------+-------------+--------+------------+-----------+--------+
|                uuid|patent_id|section_id|subsection_id|group_id| subgroup_id|   category|sequence|
+--------------------+---------+----------+-------------+--------+------------+-----------+--------+
|jyxz78olyq2fo58oy...|  6121532|         G|          G10|    G10H|  G10H1/0025|inventional|       0|
|3t723r37m4i1if7xk...|  6121532|         G|          G10|    G10H|    G10H1/00|inventional|       1|
|3bcii7zojwle05qgv...|  6121532|         G|          G10|    G10H|  G10H1/0091|inventional|       2|
|wdqqexk2781exxt0z...|  6121532|         G|          G10|    G10H|    G10H1/02|inventional|       3|
|l3ti0nnva2l56or8y...|  6121532|         G|          G10|    G10H|  G10H1/0575|inventional|       4|
|m7yrepwt46pmoj8su...|  6121532|         G|          G10|    G10H|    G10H1/20|inventional|       5|
|00346vpqo7vtf12ok...|  6121532|         G|          G10|    G10H|    G10H1/28|inventional|

In [29]:
df_cpc_group = spark.read.option("inferschema", "true").csv('data/cpc_group.tsv',
                    header=True,       # use headers or not
                    quote='"',         # char for quotes
                    sep="\t")          # char for separation  

# prints the schema
df_cpc_group.printSchema()

# some functions are still valid
print("line count: {}\n".format(df_cpc_group.count()))

# show the table in a oh-so-nice format
df_cpc_group.show()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)

line count: 671

+----+--------------------+
|  id|               title|
+----+--------------------+
|A01B|SOIL WORKING IN A...|
|A01C|PLANTING; SOWING;...|
|A01D| HARVESTING; MOWING |
|A01F|PROCESSING OF HAR...|
|A01G|HORTICULTURE; CUL...|
|A01H|NEW PLANTS OR PRO...|
|A01J|MANUFACTURE OF DA...|
|A01K|ANIMAL HUSBANDRY;...|
|A01L| SHOEING OF ANIMALS |
|A01M|CATCHING, TRAPPIN...|
|A01N|PRESERVATION OF B...|
|A21B|BAKERS' OVENS; MA...|
|A21C|MACHINES OR EQUIP...|
|A21D|TREATMENT, e.g. P...|
|A22B|        SLAUGHTERING|
|A22C|PROCESSING MEAT, ...|
|A23B|PRESERVING, e.g. ...|
|A23C|DAIRY PRODUCTS, e...|
|A23D|EDIBLE OILS OR FA...|
|A23F|COFFEE; TEA; THEI...|
+----+--------------------+
only showing top 20 rows



In [30]:
df_cpc_subgroup = spark.read.option("inferschema", "true").csv('data/cpc_subgroup.tsv',
                    header=True,       # use headers or not
                    quote='"',         # char for quotes
                    sep="\t")          # char for separation  

# prints the schema
df_cpc_subgroup.printSchema()

# some functions are still valid
print("line count: {}\n".format(df_cpc_subgroup.count()))

# show the table in a oh-so-nice format
df_cpc_subgroup.show()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)

line count: 259863

+---------+--------------------+
|       id|               title|
+---------+--------------------+
| A01B1/00|         Hand tools |
| A01B1/02|Hand tools -Spade...|
|A01B1/022|Hand tools -Spade...|
|A01B1/024|Hand tools -Spade...|
|A01B1/026|Hand tools -Spade...|
|A01B1/028|Hand tools -Spade...|
| A01B1/04|Hand tools -Spade...|
| A01B1/06|Hand tools -Hoes;...|
|A01B1/065|Hand tools -Hoes;...|
| A01B1/08|Hand tools -Hoes;...|
| A01B1/10|Hand tools -Hoes;...|
| A01B1/12|Hand tools -Hoes;...|
| A01B1/14|Hand tools -Hoes;...|
| A01B1/16|Hand tools -Tools...|
|A01B1/165|Hand tools -Tools...|
| A01B1/18|Hand tools -Tools...|
| A01B1/20|Hand tools -Combi...|
| A01B1/22|Hand tools -Attac...|
|A01B1/222|Hand tools -Attac...|
|A01B1/225|Hand tools -Attac...|
+---------+--------------------+
only showing top 20 rows



In [31]:
df_cpc_subsection = spark.read.option("inferschema", "true").csv('data/cpc_subsection.tsv',
                    header=True,       # use headers or not
                    quote='"',         # char for quotes
                    sep="\t")          # char for separation  

# prints the schema
df_cpc_subsection.printSchema()

# some functions are still valid
print("line count: {}\n".format(df_cpc_subsection.count()))

# show the table in a oh-so-nice format
df_cpc_subsection.show()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)

line count: 136

+---+--------------------+
| id|               title|
+---+--------------------+
|A01|AGRICULTURE; FORE...|
|A21|BAKING; EDIBLE DO...|
|A22|BUTCHERING; MEAT ...|
|A23|FOODS OR FOODSTUF...|
|A24|TOBACCO; CIGARS; ...|
|A41|     WEARING APPAREL|
|A42|            HEADWEAR|
|A43|            FOOTWEAR|
|A44|HABERDASHERY; JEW...|
|A45|HAND OR TRAVELLIN...|
|A46|           BRUSHWARE|
|A47|FURNITURE; DOMEST...|
|A61|MEDICAL OR VETERI...|
|A62|LIFE-SAVING; FIRE...|
|A63|SPORTS; GAMES; AM...|
|A99|SUBJECT MATTER NO...|
|B01|PHYSICAL OR CHEMI...|
|B02|CRUSHING, PULVERI...|
|B03|SEPARATION OF SOL...|
|B04|CENTRIFUGAL APPAR...|
+---+--------------------+
only showing top 20 rows



In [44]:
ddf = ddf.join(df_cpc_group.selectExpr("id", "title as group_title"), ddf.group_id == df_cpc_group.id).drop('id')

In [45]:
ddf = ddf.join(df_cpc_subgroup.selectExpr("id", "title as subgroup_title"), ddf.subgroup_id == df_cpc_subgroup.id).drop('id')

In [46]:
ddf = ddf.join(df_cpc_subsection.selectExpr("id", "title as subsection_title"), ddf.subsection_id == df_cpc_subsection.id).drop('id')

In [49]:
ddf.filter(ddf.patent_id==6121532).sort(ddf.sequence.asc()).select(["title","kind","num_claims","category","sequence","group_title","subgroup_title","subsection_title"]).show(99,False)

+-----------------------------------------------------------+----+----------+-----------+--------+---------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------